In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

import helpers as hp

%load_ext autoreload
%autoreload 2


# Download rent data

In [2]:
base_url = 'http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/'
file_names = [
    'detailed-lodged-bonds.csv',
    'detailed-mean-rents.csv',
    'detailed-geo-mean-rents.csv',
]

for file_name in file_names:
    url = base_url + file_name
    print("Getting and saving", url)
    r = requests.get(url)
    path = hp.DATA_DIR/'collected'/file_name
    with path.open('w') as tgt:
        tgt.write(r.text)


Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-lodged-bonds.csv
Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-mean-rents.csv
Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-geo-mean-rents.csv


# Prepare rent data

In [3]:
# Reshape and merge all rent data sets

def clean(f, name):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=name)
    
    return f

paths = [
    hp.DATA_DIR/'collected'/'detailed-lodged-bonds.csv',
    hp.DATA_DIR/'collected'/'detailed-mean-rents.csv',
    hp.DATA_DIR/'collected'/'detailed-geo-mean-rents.csv',
]
names = [
    'rent_count', 
    'rent_mean', 
    'rent_geo_mean',
]
frames = []
for path, name in zip(paths, names):
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, name))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = hp.get_path('au2001_csv')
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = hp.get_path('rents')
f.to_csv(path, index=False)
f[f['rent_count'].notnull()].head()


,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
244,500202,Flat or Apartment,1,1995-03-01,8.0,82.0,82.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
264,500202,Flat or Apartment,1,1996-03-01,5.0,79.0,79.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
267,500202,House,3,1996-03-01,5.0,123.0,118.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
292,500202,House,3,1997-06-01,5.0,154.0,152.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
317,500202,House,3,1998-09-01,7.0,139.0,137.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo


In [4]:
# Print latest 2 quarters
hp.get_latest_quarters(2)

['2018-03-01', '2018-06-01']

# Explore rents

In [5]:
rents = hp.get_data('rents')
print(rents['quarter'].unique())
rents.head()


['1993-03-01' '1993-06-01' '1993-09-01' '1993-12-01' '1994-03-01'
 '1994-06-01' '1994-09-01' '1994-12-01' '1995-03-01' '1995-06-01'
 '1995-09-01' '1995-12-01' '1996-03-01' '1996-06-01' '1996-09-01'
 '1996-12-01' '1997-03-01' '1997-06-01' '1997-09-01' '1997-12-01'
 '1998-03-01' '1998-06-01' '1998-09-01' '1998-12-01' '1999-03-01'
 '1999-06-01' '1999-09-01' '1999-12-01' '2000-03-01' '2000-06-01'
 '2000-09-01' '2000-12-01' '2001-03-01' '2001-06-01' '2001-09-01'
 '2001-12-01' '2002-03-01' '2002-06-01' '2002-09-01' '2002-12-01'
 '2003-03-01' '2003-06-01' '2003-09-01' '2003-12-01' '2004-03-01'
 '2004-06-01' '2004-09-01' '2004-12-01' '2005-03-01' '2005-06-01'
 '2005-09-01' '2005-12-01' '2006-03-01' '2006-06-01' '2006-09-01'
 '2006-12-01' '2007-03-01' '2007-06-01' '2007-09-01' '2007-12-01'
 '2008-03-01' '2008-06-01' '2008-09-01' '2008-12-01' '2009-03-01'
 '2009-06-01' '2009-09-01' '2009-12-01' '2010-03-01' '2010-06-01'
 '2010-09-01' '2010-12-01' '2011-03-01' '2011-06-01' '2011-09-01'
 '2011-12-

,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [8]:
# Slice in time and aggregate 
agg_rents = hp.aggregate_rents(rents, '2018-06-01')
agg_rents.head()

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


,rental_area,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
0,Addington,1,Canterbury,73.0,211.000000,212.000000,Christchurch City
1,Addington,2,Canterbury,35.0,315.445976,324.371429,Christchurch City
2,Addington,3,Canterbury,31.0,428.681384,434.258065,Christchurch City
3,Addington,4,Canterbury,0.0,NaN,NaN,Christchurch City
4,Addington,5+,Canterbury,0.0,NaN,NaN,Christchurch City


In [9]:
f = hp.aggregate_rents(rents, '2018-06-01', groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
f[cond]

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


,au2001,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
287,505300,1,Auckland,0.0,NaN,NaN,Rodney District
288,505300,2,Auckland,0.0,NaN,NaN,Rodney District
289,505300,3,Auckland,8.0,450.000000,451.000000,Rodney District
290,505300,4,Auckland,5.0,473.000000,473.000000,Rodney District
291,505300,5+,Auckland,0.0,NaN,NaN,Rodney District
292,505400,1,Auckland,0.0,NaN,NaN,Rodney District
293,505400,2,Auckland,0.0,NaN,NaN,Rodney District
294,505400,3,Auckland,0.0,NaN,NaN,Rodney District
295,505400,4,Auckland,0.0,NaN,NaN,Rodney District
296,505500,1,Auckland,7.0,289.000000,293.000000,Rodney District


In [10]:
# What fraction of rental data do we have by #bedrooms?

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_mean'].count()/group['rent_mean'].shape[0]
    return pd.Series(d)

date = '2018-06-01'
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())

f = hp.aggregate_rents(rents, date, groupby_cols=('rental_area', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())


/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


census area units
  #bedrooms  hit_frac
0         1  0.254658
1         2  0.495652
2         3  0.763533
3         4  0.389049
4        5+  0.063694
rental area units
  #bedrooms  hit_frac
0         1  0.585859
1         2  0.878788
2         3  1.000000
3         4  0.727273
4        5+  0.151515


# Select latest two quarters and slice into regional chunks

In [11]:
# Get latest two quarters of rents
start_date = hp.get_latest_quarters(2)[0]
rents = hp.get_data('rents')
rents = rents[rents['quarter'] >= start_date].copy()

# Create regional slices
for region in hp.REGIONS:
    # Build rents
    region_c = region.capitalize()
    region_rents = rents[rents['region'] == region_c].copy()
    print(region, region_rents['quarter'].unique(), '#rows =', region_rents.shape[0])
    path = hp.get_path('rents', region)
    print('  Saving to', path)
    
    # Create region directory if it does not exist
    if not path.parent.exists():
        path.parent.mkdir()
        
    region_rents.to_csv(path, index=False)
    

auckland ['2018-03-01' '2018-06-01'] #rows = 5544
  Saving to /home/araichev/affordability_nz/data/processed/auckland/rents.csv
canterbury ['2018-03-01' '2018-06-01'] #rows = 2540
  Saving to /home/araichev/affordability_nz/data/processed/canterbury/rents.csv
wellington ['2018-03-01' '2018-06-01'] #rows = 2408
  Saving to /home/araichev/affordability_nz/data/processed/wellington/rents.csv


# JSONize regional rents for the web, grouping by rental area and number of bedrooms

In [12]:
for region in hp.REGIONS:
    region_rents = hp.get_data('rents', region)
    d = hp.build_json_rents(region_rents)
    path = hp.get_path('rents_json', region)
    print('**', region, '\n', d)
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


** auckland 
 {'Central West': {'3': 621.0, '4': None, '1': 399.0, '2': 520.0}, 'Beachhaven/Birkdale': {'3': 551.0, '4': 627.0, '1': None, '2': 428.0}, 'Rewiti/Kumeu/Riverhead': {'3': 594.0, '4': 750.0, '1': None, '2': 450.0}, 'Rothesay/Murrays/Mairangi Bays': {'3': 648.0, '4': 803.0, '1': 241.0, '2': 486.0}, 'Papatoetoe South': {'3': 538.0, '4': None, '1': 193.0, '2': 435.0}, 'Manurewa North': {'3': 491.0, '4': 586.0, '1': 334.0, '2': 423.0}, 'Chatswood/Birkenhead/Northcote Point': {'3': 665.0, '4': 788.0, '1': 363.0, '2': 504.0}, 'Torbay': {'3': 602.0, '4': 743.0, '1': None, '2': 483.0}, 'Titirangi': {'3': 555.0, '4': 584.0, '1': 223.0, '2': None}, 'Grey Lynn/Arch Hill': {'3': 820.0, '4': 985.0, '1': 387.0, '2': 666.0}, 'Massey/Royal Heights': {'3': 504.0, '4': 602.0, '1': None, '2': 428.0}, 'Glendene': {'3': 505.0, '4': 566.0, '1': None, '2': 435.0}, 'Glen Eden': {'3': 478.0, '4': 532.0, '1': None, '2': 496.0}, 'Kingsland': {'3': 679.0, '4': 849.0, '1': 406.0, '2': 538.0}, 'Greenhit

** wellington 
 {'Taita/Naenae': {'3': 427.0, '4': None, '1': 223.0, '2': 336.0}, 'Kilbirnie/Lyall Bay': {'3': 583.0, '4': None, '1': 322.0, '2': 457.0}, 'Karori': {'3': 591.0, '4': 842.0, '1': 312.0, '2': 416.0}, 'Totara Park/Maoribank/Te Marua': {'3': 452.0, '4': None, '1': None, '2': 269.0}, 'Hutt Central/Waterloo': {'3': 544.0, '4': 666.0, '1': 247.0, '2': 428.0}, 'Paraparaumu/Raumati': {'3': 440.0, '4': 539.0, '1': 191.0, '2': 364.0}, 'Moera/Waiwhetu': {'3': 497.0, '4': 573.0, '1': None, '2': None}, 'Titahi Bay/Onepoto/Elsdon': {'3': 431.0, '4': None, '1': 237.0, '2': 397.0}, 'Paramata/Mana/Pukerua Bay': {'3': 522.0, '4': 605.0, '1': 293.0, '2': 354.0}, 'Waikanae/Otaki': {'3': 408.0, '4': 423.0, '1': None, '2': 348.0}, 'Western Hills/Haywards': {'3': 507.0, '4': 594.0, '1': 215.0, '2': 414.0}, 'Mt Victoria/Roseneath': {'3': 670.0, '4': 784.0, '1': 331.0, '2': 540.0}, 'Northland': {'3': 628.0, '4': 585.0, '1': 334.0, '2': 479.0}, 'Petone/Esplanade': {'3': 537.0, '4': 672.0, '1': 25